In [1]:
## --- Fast Import Snippet ---##
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys, os
import math, random
import pathlib
import time
import warnings
import itertools
import inspect
import numbers, string
import collections
from collections import namedtuple, OrderedDict
from pprint import pprint
from tqdm import tqdm

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import SimpleITK as sitk
from PIL import Image

import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader
import torchvision.transforms as T
import albumentations as A
from skimage.transform import resize as nd_resize
from scipy.ndimage import rotate as nd_rotate

np.set_printoptions(precision=3)
curr_path = pathlib.Path(os.getcwd()).parent.absolute()

cards = !echo $SGE_HGR_gpu_card
device = torch.device(f"cuda:{cards[0]}" if torch.cuda.is_available() else 'cpu')
print(device)


cuda:1


In [6]:
# Load Config
from run import get_experiment_cfg
config = get_experiment_cfg('prevec_general.yaml')

# Get Data Components
from experiments.prevec.data_setup import get_pretrain_data_components
data_d = get_pretrain_data_components(config)

[CFG] Given cfg file "prevec_general.yaml" does not exist! Searching for matching name in experiment's config folder..
 ✔ Cfg experiment matched at /afs/crc.nd.edu/user/y/yzhang46/_3DPRE/src/experiments/prevec/configs.
 > Loading config (prevec_general.yaml).. done.
🖼️  Getting MMWHS, BCV, MSD Liver, MSD Lung samples.
[Took 0.50 sec for 231 samples.]
🖼️  Task: mg, Config: {'t_mg_flip': 0.4, 't_mg_shuffle': 0.5, 't_mg_nonlinear': 0.9, 't_mg_paint': 0.9, 't_mg_inpaint': 0.8}.

Train Data Components:
🖼️  Sampling from scales: [0.5, 2]


In [ ]:
loader = data_d['train_loader']
for it, batch in enumerate(loader):
    sample = sample['samples']
    X = batch['X']
    Y = batch['Y']
    